In [3]:
import fitz  # PyMuPDF

# PDF 파일 열기
pdf_path = r"C:\Jimin\cg_DeltaLaw\data\raw\2205429_의사국 의안과_의안원문.pdf"
doc = fitz.open(pdf_path)

# 모든 페이지에서 텍스트 추출
text_pages = [page.get_text() for page in doc]
full_text = "\n".join(text_pages)


In [4]:
# 신구조문 구간 추출 로직 (예: "현 행" ~ "개 정 안")
import re

# "현 행"과 "개 정 안" 섹션 구간을 정규표현식으로 탐지
pattern = r"(현\s*행.*?)(개\s*정\s*안.*?)(제\s*\d+조|$)"
matches = re.findall(pattern, full_text, flags=re.DOTALL)

parsed_sections = []
for match in matches:
    current, revised, next_start = match
    parsed_sections.append({
        "현행": current.strip(),
        "개정안": revised.strip()
    })


In [6]:
import fitz  # PyMuPDF
import re
import pandas as pd

pdf_path = r"C:\Jimin\cg_DeltaLaw\data\raw\2205429_의사국 의안과_의안원문.pdf"

doc = fitz.open(pdf_path)
text_pages = [page.get_text() for page in doc]
full_text = "\n".join(text_pages)

# 신구조문 영역 추출
pattern = r"(현\s*행.*?)(개\s*정\s*안.*?)(제\s*\d+조|$)"
matches = re.findall(pattern, full_text, flags=re.DOTALL)

# 리스트 → DataFrame 변환
df = pd.DataFrame([
    {"현행": m[0].strip(), "개정안": m[1].strip()} for m in matches
])


In [8]:
# 화면에 보기
print(df.head())

# 파일로 저장
df.to_excel(r"C:\Jimin\cg_DeltaLaw\data\processed\신구조문대비표_결과.xlsx", index=False)

                                                  현행      개정안
0  현행법상법정자본금은30조원이며, 2\n024년9월현재납입자본금잔액은26.31조원으로...  개\n정\n안


In [10]:
df.to_csv(r"C:\Jimin\cg_DeltaLaw\data\processed\신구조문대비표_결과.csv", index=False, encoding="utf-8-sig")

In [12]:
import pdfplumber
import pandas as pd

# PDF 경로
pdf_path = r"C:\Jimin\cg_DeltaLaw\data\raw\2205429_의사국 의안과_의안원문.pdf"

# 신구조문대비표가 있는 페이지 추출 + 표 추출 시도
extracted_rows = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row and any("현행" in str(cell) or "개정안" in str(cell) for cell in row):
                    extracted_rows.append(("page", i + 1))
                extracted_rows.append(row)

# 데이터프레임 변환 (최대 3열까지로 가정)
df = pd.DataFrame([r for r in extracted_rows if isinstance(r, list)], columns=["현행", "개정안", "비고"][:len(max(extracted_rows, key=lambda x: len(x) if isinstance(x, list) else 0))])

# 화면에 보기
print(df.head())
df.to_excel(r"C:\Jimin\cg_DeltaLaw\data\processed\신구조문대비표_결과.xlsx", index=False)



                                                  현행  \
0                                           의 안\n번 호   
1                                                현 행   
2  제5조(자본금) ① 한국산업은행의\n자본금은 30조원 이내에서 정\n관으로 정하되,...   

                                                 개정안  
0                                               5429  
1                                              개 정 안  
2  제5조(자본금) ① --------------\n--------60조원-------...  


In [13]:
import pdfplumber
import re
import pandas as pd

pdf_path = r"C:\Jimin\cg_DeltaLaw\data\raw\2205429_의사국 의안과_의안원문.pdf"

def extract_change_rows(pdf_path):
    changes = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                for row in table:
                    if not row or len(row) < 2:
                        continue
                    old, new = row[0], row[1]
                    if not old or not new:
                        continue

                    # 조문 정보
                    match = re.search(r"(제\d+조)\s*(\(?\d+\)?항)?", old)
                    if not match:
                        continue
                    clause = match.group(1)
                    paragraph = match.group(2) if match.group(2) else ""

                    # 숫자값 추출 (억, 조 단위 포함)
                    old_num = re.search(r"(\d+[조억]?)", old)
                    new_num = re.search(r"(\d+[조억]?)", new)

                    if old_num and new_num and old_num.group() != new_num.group():
                        changes.append({
                            "조문": clause,
                            "항": paragraph.strip(),
                            "기존": old_num.group(),
                            "개정": new_num.group()
                        })

    return pd.DataFrame(changes)

df = extract_change_rows(pdf_path)
df.to_csv("신구조문_수치변경.csv", index=False, encoding="utf-8-sig")
print("✅ 저장 완료: 신구조문_수치변경.csv")


✅ 저장 완료: 신구조문_수치변경.csv


In [14]:
df.to_excel(r"C:\Jimin\cg_DeltaLaw\data\processed\신구조문_수치변경.xlsx", index=False)